In [ ]:
import uproot as up
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
sns.set(font_scale=1.5)
sns.set_style('white')

In [ ]:
rcParams.update({'figure.autolayout':True})
rcParams.update({'figure.figsize':[15,8]})

In [ ]:
input_dir="./PXE05M/"
# PX1M_Th228_pos648.5_0_-1022.6.root

In [ ]:
def get_filtered_tuple_from_root(filename):
    file=up.open(filename)
    tree=file['tree']
    energy,passed_xy_thresh,passed_z_thresh,n_x_ch_abovenoise,\
    n_y_ch_abovenoise,m_nOPCal,m_nQ,NESTBugFound,m_DNNvalue,standoff,weight=\
    tree.arrays(['energy','passed_xy_thresh','passed_z_thresh','n_x_ch_abovenoise',\
                 'n_y_ch_abovenoise','m_nOPCal','m_nQ','NESTBugFound','m_DNNvalue','standoff','weight'],outputtype=tuple)
    sel_filter=(passed_xy_thresh & passed_z_thresh & (n_x_ch_abovenoise>0) & (n_y_ch_abovenoise > 0) & (m_nOPCal < 1.064 * m_nQ+703) & \
                (m_nOPCal>0.644 * m_nQ - 2411) & (~NESTBugFound) & (m_DNNvalue>0.85) & (standoff>100))
    return energy[sel_filter],weight[sel_filter]

In [ ]:
#root_file='PX1M_Th228_pos648.5_0_-1022.6.root'
root_file='PXE05M_Th228_pos648.5_0_-1022.6.root'


In [ ]:
ene,weight=get_filtered_tuple_from_root(input_dir+root_file)

In [ ]:
component_lists = [
['TPCVessel', 'SiPMSubstrate', 'SiPMStave', 'SiPMModule'],
['HVTube', 'HVPlunger', 'HVFeedthruCore', 'HVFeedthruBox', 'HVCable'],
['ChargeModuleSupport', 'ChargeModuleBacking', 'Cathode', ],
['FieldRing', 'SupportRodsAndSpacers', 'Bulge'],
['HFE','CalibrationGuideTube1', 'CalibrationGuideTube2','TPCSupport'],
['RepeaterBoxShell', 'InnerCryostat', 'InnerCryostatSupport', 'InnerCryostatLiner']
]

In [ ]:
isotpes=['Th228']
for isotope in ['U238', 'Th232']:
    for c_list in component_lists:
        for component in c_list:
            filename = f"{input_dir}Baseline2019_{isotope}_{component}.root"
            ene, weight = get_filtered_tuple_from_root(filename)
            plt.hist(ene, bins=200, histtype=u'step', weights=weight, density=True, label=component)
        plt.yscale("log")
        plt.ylabel("Counts")
        plt.xlabel("Energy [keV]")
        plt.title(f"{isotope} Energy PDF for SS events in the inner 2000kg")
        plt.legend()
        plt.show()